<a href="https://colab.research.google.com/github/Eparcham/PageRankBigDate/blob/master/PageRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from scipy.linalg import eig

useEgn = True
## read file
dir_ = "graph.csv"
graph = pd.read_csv(dir_, header = None, sep = ',').to_numpy()
print(graph)
b=np.linalg.det(graph)
print(b)

n = len(graph)
print("number of Node: ", n)

iter_num = 100000 ## step of walk
teleport_prob = 0.1 # for walk random in pages

## preprocessing (make graph like sparse matrix)
print("*"*50)
processed_graph = []
for i in range(n):
    processed_graph.append([])
    for j in range(n):
        if graph[i,j] ==1:
            processed_graph[-1].append(j)

print(processed_graph)
## call fun take one step from cur_pos and return the new pos
cur_pos = np.random.randint(n) ## initial pos of random walk
def one_step():
    if len(processed_graph[cur_pos]) == 0: # dead end than we must teleport
       return np.random.randint(n)

    if np.random.rand()<teleport_prob: # make teleport  we can use random walk prob here
        return np.random.randint(n)
    else:
        n_options = len(processed_graph[cur_pos]) ## give number of node we can go
        return processed_graph[cur_pos][np.random.randint(n_options)]


pagerank = np.zeros([1,n])
for i in range(iter_num):
    cur_pos = one_step()
    pagerank[0,cur_pos] +=1

print(pagerank)  ## show vist page
pagerank = pagerank / iter_num
print(pagerank)  ## show prob page

if useEgn :
    ## coputation of pagerank using eigenvectors
    def prob(a,b):
        if len(processed_graph[a])==0: ## dead end
            return 1/n  ## go from a to b just with teleport
        else:
            if b in processed_graph[a]: ## we can go fram a to b or graph[a,b]==1
                return  teleport_prob*(1/n) + (1 - teleport_prob) * 1/(len(processed_graph[a]))
            else:
                return teleport_prob * (1/n)  # we can go to b just with teleport and with teleport prob

    M = np.zeros([n,n])
    for i in range(n):
        for j in range(n):
            M[i,j] = prob(i,j)
    # print(M)
    print(np.sum(M,axis=1))

    ## CAL EIG
    eig_vals, eig_vecs  = eig(M,left =True, right=False)
    idx = np.argmax(eig_vals)
    fast_pagerank = eig_vecs[:,idx]/np.sum(eig_vecs[:,idx])
    print(np.abs(fast_pagerank))





[[0 1 1]
 [1 0 1]
 [0 0 0]]
0.0
number of Node:  3
**************************************************
[[1, 2], [0, 2], []]
[[29008. 28966. 42026.]]
[[0.29008 0.28966 0.42026]]
[1. 1. 1.]
[0.28985507 0.28985507 0.42028986]


In [2]:
import pandas as pd
import numpy as np
from scipy.linalg import eig
import scipy as sp
import scipy.sparse as sprs
import scipy.spatial
import scipy.sparse.linalg
useEgn = True
## read file
dir_ = "graph.csv"
graph = pd.read_csv(dir_, header = None, sep = ',').to_numpy()
print(graph)
## show how to work @
# nCol = 5
# zarib1 = 2
# zarib2 =2
# sum_ = (zarib1*zarib2) * nCol
# a = np.ones([3,nCol])*zarib1
# b = np.ones([nCol,1])*zarib2
# c = a @ b
# print(c)

def pagerank_power(A, max_iter=1000,tol=1e-06):

    n, _ = A.shape
    x = np.asarray(A.sum(axis=1)).reshape(-1)
    k = x.nonzero()[0]
    D_1 = sprs.csr_matrix((1 / x[k], (k, k)), shape=(n, n))
    z_T = (((x != 0) + (x == 0)) / n)[sp.newaxis, :]
    M =  A.T @ D_1

    r = np.ones([n,1])
    r = r / n
    oldx = np.zeros((n, 1))

    iteration = 0
    while sp.linalg.norm(r - oldx) > tol:
        oldx = r
        ## OLD WAY
        # x = M @ x + s @ (z_T @ x)
        # x = M @ x + (z_T @ x)
        # x = np.dot(M, x) + np.dot(s , np.dot(z_T , x))
        r = np.dot(M, r) + np.dot(z_T, r)
        iteration += 1
        if iteration >= max_iter:
            break

    r = r / sum(r)

    return r.reshape(-1)

print(pagerank_power(graph))



[[0 1 1]
 [1 0 1]
 [0 0 0]]
[0.30195628 0.30195628 0.39608744]
